In [3]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import itertools
import os
import numpy as np

# Define the input directory and output file path
input_dir = r'C:\Users\skanigir\epa\Scripts\notebooks\GRA\Cambium\Main\2023\data\CO2\georgia\harmonicmean\2025'
output_file_path = r'C:\Users\skanigir\epa\Scripts\notebooks\GRA\Cambium\Main\2023\data\CO2\georgia\harmonicmean\results\forecasted_results_new.xlsx'

# Scan for all .xlsx files in the input directory
files = [f for f in os.listdir(input_dir) if f.endswith('.xlsx')]

# Initialize an Excel writer
writer = pd.ExcelWriter(output_file_path, engine='xlsxwriter')

# Initialize a DataFrame to store the performance metrics
metrics_columns = ['File', 'MSE', 'RMSE', 'MAE', 'R²', 'MAPE']
metrics_df = pd.DataFrame(columns=metrics_columns)

# Process each file
for file_name in files:
    file_path = os.path.join(input_dir, file_name)
    
    try:
        # Attempt to read the Excel file
        data = pd.read_excel(file_path, engine='openpyxl')
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        continue
    
    # Prepare the data for Prophet model
    prophet_data = data.melt(id_vars=["Month"], var_name="Year", value_name="Emission")
    prophet_data['ds'] = pd.to_datetime(prophet_data['Year'].astype(str) + '-' + prophet_data['Month'].astype(str) + '-01')
    prophet_data = prophet_data[['ds', 'Emission']].dropna().rename(columns={'Emission': 'y'})
    
    # Normalize the Emission data
    scaler = MinMaxScaler()
    prophet_data['y'] = scaler.fit_transform(prophet_data[['y']])
    
    # Define the grid search parameters
    param_grid = {
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5, 1.0],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative']
    }
    
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    
    # Initialize variables to store the best parameters and the corresponding MSE
    best_params = None
    best_mse = float('inf')
    
    # Grid search to find the best parameters
    for params in all_params:
        model = Prophet(**params)
        model.fit(prophet_data)
        
        # Generate future dates for forecasting until 2050
        future_dates = pd.date_range(start='2025-01-01', end='2050-12-01', freq='MS')
        future_dates = pd.DataFrame(future_dates, columns=['ds'])
        
        # Forecast emissions for the future dates
        forecast = model.predict(future_dates)
        forecast['yhat'] = forecast['yhat'].apply(lambda x: max(x, 0))  # Ensure no negative predictions
        
        # Merge the forecast with the actual data for comparison
        forecasted = forecast.set_index('ds').loc[prophet_data['ds'], 'yhat']
        prophet_data['yhat'] = forecasted.values
        
        # Calculate the Mean Squared Error (MSE)
        mse = mean_squared_error(prophet_data['y'], prophet_data['yhat'])
        
        # Update the best parameters if the current MSE is lower
        if mse < best_mse:
            best_params = params
            best_mse = mse
    
    # Print the best parameters and the corresponding MSE
    print(f'Best Parameters for {file_name}: {best_params}')
    print(f'Best MSE for {file_name}: {best_mse}')
    
    # Train the final model with the best parameters
    model = Prophet(**best_params)
    model.fit(prophet_data)
    
    # Generate future dates for forecasting until 2050
    future_dates = pd.date_range(start='2025-01-01', end='2050-12-01', freq='MS')
    future_dates = pd.DataFrame(future_dates, columns=['ds'])
    
    # Forecast emissions for the future dates
    forecast = model.predict(future_dates)
    forecast['yhat'] = forecast['yhat'].apply(lambda x: max(x, 0))  # Ensure no negative predictions
    
    # Inverse transform the normalized predictions and actuals
    forecast['yhat'] = scaler.inverse_transform(forecast[['yhat']])
    prophet_data['yhat'] = scaler.inverse_transform(prophet_data[['yhat']].values.reshape(-1, 1))
    prophet_data['y'] = scaler.inverse_transform(prophet_data[['y']].values.reshape(-1, 1))
    
    # Merge the forecast with the actual data for comparison
    forecasted = forecast.set_index('ds').loc[prophet_data['ds'], 'yhat']
    prophet_data['yhat'] = forecasted.values
    
    # Calculate deviation and deviation percentage
    prophet_data['deviation'] = prophet_data['y'] - prophet_data['yhat']
    
    # Avoid division by zero and high MAPE values due to very small actual values
    prophet_data['deviation_percentage'] = np.where(
        prophet_data['y'].abs() < 1e-10, 0, (prophet_data['deviation'] / prophet_data['y']) * 100
    )
    
    # Calculate performance metrics
    mse = mean_squared_error(prophet_data['y'], prophet_data['yhat'])
    rmse = mse ** 0.5
    mae = mean_absolute_error(prophet_data['y'], prophet_data['yhat'])
    r2 = r2_score(prophet_data['y'], prophet_data['yhat'])
    mape = (prophet_data['deviation_percentage'].abs().mean())
    
    # Print performance metrics
    print(f'Mean Squared Error (MSE) for {file_name}: {mse:.2f}')
    print(f'Root Mean Squared Error (RMSE) for {file_name}: {rmse:.2f}')
    print(f'Mean Absolute Error (MAE) for {file_name}: {mae:.2f}')
    print(f'R-squared (R²) for {file_name}: {r2:.2f}')
    print(f'Mean Absolute Percentage Error (MAPE) for {file_name}: {mape:.2f}%')
    
    # Save the performance metrics to the DataFrame
    new_metrics = pd.DataFrame([{
        'File': file_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE': mape
    }])
    metrics_df = pd.concat([metrics_df, new_metrics], ignore_index=True)
    
    # Save the results to the Excel sheet
    sheet_name_actual = file_name.replace('harmonicmean', 'hm_actual').replace('.xlsx', '')[:31]
    sheet_name_forecast = file_name.replace('harmonicmean', 'hm_forecast').replace('.xlsx', '')[:31]
    
    actual_values = prophet_data[['ds', 'y']]
    actual_values.to_excel(writer, sheet_name=sheet_name_actual, index=False)
    
    forecasted_values = forecast[['ds', 'yhat']]
    forecasted_values.to_excel(writer, sheet_name=sheet_name_forecast, index=False)

# Save the performance metrics to a separate sheet
metrics_df.to_excel(writer, sheet_name='Performance_Metrics', index=False)

# Save the workbook
writer.close()

print(f'Forecasted results and performance metrics saved to {output_file_path}')


16:57:37 - cmdstanpy - INFO - Chain [1] start processing
16:57:37 - cmdstanpy - INFO - Chain [1] done processing
16:57:37 - cmdstanpy - INFO - Chain [1] start processing
16:57:37 - cmdstanpy - INFO - Chain [1] done processing
16:57:38 - cmdstanpy - INFO - Chain [1] start processing
16:57:38 - cmdstanpy - INFO - Chain [1] done processing
16:57:38 - cmdstanpy - INFO - Chain [1] start processing
16:57:39 - cmdstanpy - INFO - Chain [1] done processing
16:57:39 - cmdstanpy - INFO - Chain [1] start processing
16:57:39 - cmdstanpy - INFO - Chain [1] done processing
16:57:39 - cmdstanpy - INFO - Chain [1] start processing
16:57:40 - cmdstanpy - INFO - Chain [1] done processing
16:57:40 - cmdstanpy - INFO - Chain [1] start processing
16:57:40 - cmdstanpy - INFO - Chain [1] done processing
16:57:40 - cmdstanpy - INFO - Chain [1] start processing
16:57:41 - cmdstanpy - INFO - Chain [1] done processing
16:57:41 - cmdstanpy - INFO - Chain [1] start processing
16:57:41 - cmdstanpy - INFO - Chain [1]

Best Parameters for harmonicmean_Mid-case_CO2_Emissions.xlsx: {'changepoint_prior_scale': 1.0, 'seasonality_prior_scale': 1.0, 'seasonality_mode': 'multiplicative'}
Best MSE for harmonicmean_Mid-case_CO2_Emissions.xlsx: 7.840477851008037e-09


16:57:55 - cmdstanpy - INFO - Chain [1] done processing


Mean Squared Error (MSE) for harmonicmean_Mid-case_CO2_Emissions.xlsx: 0.00
Root Mean Squared Error (RMSE) for harmonicmean_Mid-case_CO2_Emissions.xlsx: 0.01
Mean Absolute Error (MAE) for harmonicmean_Mid-case_CO2_Emissions.xlsx: 0.01
R-squared (R²) for harmonicmean_Mid-case_CO2_Emissions.xlsx: 1.00
Mean Absolute Percentage Error (MAPE) for harmonicmean_Mid-case_CO2_Emissions.xlsx: 0.00%
Forecasted results and performance metrics saved to C:\Users\skanigir\epa\Scripts\notebooks\GRA\Cambium\Main\2023\data\CO2\georgia\harmonicmean\results\forecasted_results_new.xlsx


C:\Users\skanigir\AppData\Local\Temp\ipykernel_17108\3829563206.py:138: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, new_metrics], ignore_index=True)
